# Support Vector Machine

In [5]:
import pandas as pd
import cv2
from skimage.filters import prewitt_h, prewitt_v, sobel, roberts
from skimage.feature import canny
from skimage.io import imshow as ski_imshow
from matplotlib import pyplot as plt
import numpy
import os
from IPython.display import display
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, KFold # Import train_test_split, GridSearchCV, and KFold functions and classses
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn import svm

# Import TensorFlow
import tensorflow as tf
from keras import datasets, layers, models
import matplotlib.pyplot as plt
from projectLib import getLabelArray

## Getting Data

In [3]:
data = pd.read_csv("..\data\HAM10000_metadata.csv")

# Creating a dataframe with 10%
# values of original dataframe
testData = data.sample(frac = 0.1)
 
# Creating dataframe with
# rest of the 84% values
trainData = data.drop(testData.index)

# class labels
classes = [['mel', 'Melanoma'], ['nv', 'Melanocytic nevus'], ['bcc', 'Basal cell carcinoma'], 
           ['akiec', 'Actinic keratosis / Bowen’s disease'], ['bkl', 'Benign keratosis'], ['df', 'Dermatofibroma'],
           ['vasc', 'Vascular lesion']]


# row = data.iloc[1]:
testImages = []
testLabels = []
for index, row in testData.iterrows():
    if index % 1000 == 0:
        print (index)
    img = cv2.imread("..\data\\allData\HAM10000_images\\" + row['image_id'] +".jpg", flags= cv2.IMREAD_GRAYSCALE)
    # img = img / 255  ## changes values 0-1
    dim = (108, 81)
    img = cv2.resize(img, dim)  ## resize
    img = canny(img, sigma=.5)
    testImages.append(numpy.reshape(img, 108*81))
    arr = None
    if row['dx'] == 'mel':
        arr =  0
    elif row['dx'] == 'nv':
        arr =  1
    elif row['dx'] == 'bcc':
        arr =  2
    elif row['dx'] == 'akiec':
        arr =  3
    elif row['dx'] == 'bkl':
        arr =  4
    elif row['dx'] == 'df':
        arr =  5
    else:
        arr =  6
    testLabels.append(arr)

trainImages = []
trainLabels = []
for index, row in trainData.iterrows():
    if index % 1000 == 0:
        print (index)
    img = cv2.imread("..\data\\allData\HAM10000_images\\" + row['image_id'] +".jpg", flags= cv2.IMREAD_GRAYSCALE)
    # img = img / 255  ## changes values 0-1
    dim = (108, 81)
    img = cv2.resize(img, dim)  ## resize
    img = canny(img, sigma=.5)
    trainImages.append(numpy.reshape(img, 108*81))
    arr = None
    if row['dx'] == 'mel':
        arr =  0
    elif row['dx'] == 'nv':
        arr =  1
    elif row['dx'] == 'bcc':
        arr =  2
    elif row['dx'] == 'akiec':
        arr =  3
    elif row['dx'] == 'bkl':
        arr =  4
    elif row['dx'] == 'df':
        arr =  5
    else:
        arr =  6
    trainLabels.append(arr)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


## Create/ Train Model

In [4]:
xTrain = numpy.array(trainImages)
yTrain = numpy.array(trainLabels)
xTest = numpy.array(testImages)
yTest = numpy.array(testLabels)

In [6]:
# Grid Search

parameters = [{'C':[1, 10, 100], 'kernel':['poly'], 'degree':[2, 3, 4], 'decision_function_shape':['ovo']},
              {'C':[1, 10, 100], 'kernel':['rbf'], 'decision_function_shape':['ovo']},
              
]


baseSVC = svm.SVC()
gridCLF = GridSearchCV(baseSVC, parameters)
gridCLF.fit(xTrain, yTrain)

print(gridCLF.best_estimator_)
print(gridCLF.best_score_)
print(gridCLF.best_params_)


In [9]:
# Test with k-fold cross validation

kModel = svm.SVC( kernel= 'rbf' , decision_function_shape='ovo')

kFold = KFold(n_splits=5, shuffle=True)

scores = cross_validate(kModel, xTrain, yTrain, cv=kFold)

print(scores['test_score'].mean)

In [10]:
# create svm

# model = svm.SVC( kernel= 'linear' , decision_function_shape='ovo')   ## meth 1   56.08

# model = svm.SVC( kernel= 'rbf' , decision_function_shape='ovo')   ## meth 2 69.8  

model = svm.SVC( kernel= 'poly' , decision_function_shape='ovo') ## 69.4



# train svm classifier
model.fit(X=xTrain, y= yTrain)
# xTrain.shape
# yTrain.shape

SVC(decision_function_shape='ovo', kernel='poly')

In [11]:

svm_predictions = model.predict(xTest)
  
# model accuracy for X_test  
accuracy = model.score(xTest, yTest)

print(accuracy)
  
# creating a confusion matrix
# cm = svm.confusion_matrix(yTest, svm_predictions)

0.6946107784431138
